# INFO 2950 Final Project

In [1]:
pip install linearmodels

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Imports
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests
from bs4 import BeautifulSoup
import os
import re
import pdfplumber
import duckdb
import statsmodels.api as sm
from io import BytesIO
from linearmodels.panel import PanelOLS

## Introduction:

The rapid adoption of electric vehicles (EVs) is pivotal in reducing greenhouse gas emissions and transitioning towards sustainable transportation. As the U.S. pushes towards a greener future, understanding what drives EV adoption is becoming increasingly important. Research shows that federal incentives like tax credits have historically played a critical role in encouraging EV adoption (U.S. DOE, 2023), while renewable energy availability has been linked to infrastructure expansion and greater adoption rates (Energy Information Administration, 2022). This project seeks to explore key determinants of state-level EV registrations, including federal incentives, renewable energy production, EV pricing, and charging infrastructure availability, to build predictive models for EV registration growth.

Our primary research question is: ***What factors reliably predict EV registration growth across U.S. states, and how do these vary by region?*** To address this question, we hypothesize that (1) EV registrations increase with the total number of federal incentives and (2) EV registrations are positively correlated with a state's renewable energy production. Both hypotheses are tested using state-level data over multiple years, employing linear regression models to evaluate the impact of these variables on EV adoption trends.

To make this analysis focused and actionable, we narrowed the scope of our project to five states: **California, Florida, Texas, Washington, and New Jersey**. These states were selected because they lead the nation in both EV registrations and growth from 2016 to 2023. By studying these key states, we aim to identify patterns and strategies that could potentially be scaled to other regions in the future.

The data for this analysis is drawn from publicly available sources, including the U.S. Department of Energy's Alternative Fuels Data Center, the Energy Information Administration, U.S. Census Bureau, and Open Charge Map. The dataset includes national-level variables (e.g., federal EV incentives, EV pricing) and state-level variables (e.g., charging station availability, renewable energy production). To ensure fair comparisons, we calculated EV registrations per capita, which accounts for population differences across states.

Preliminary trends suggest that []. By identifying the influential factors driving this growth, our predictive model aims to guide policymakers and stakeholders in designing effective strategies to accelerate EV adoption. This work not only sheds light on current trends but also provides actionable insights to support smarter policies, targeted infrastructure investments, and a smoother transition to a cleaner, more sustainable transportation future.

## Data Description and Cleaning:

In [6]:
# Clean, merged data file
data = pd.read_csv('final_ev_dataset.csv')

display(data.head(n=5))

print(data.isnull().sum())

,year,state,registration,population,reg_per_capita,affordability,incentives,renewable_prod,charge_stations,charge_per_capita
0,2016,California,141500.0,39167117,0.003613,22995.0,20,527.599976,1413,0.000036
1,2016,Florida,11600.0,20613477,0.000563,22995.0,20,480.700012,381,0.000018
2,2016,New Jersey,4200.0,8870827,0.000473,22995.0,20,334.600006,127,0.000014
3,2016,Texas,11900.0,27914410,0.000426,22995.0,20,427.399994,334,0.000012
4,2016,Washington,14900.0,7294771,0.002043,22995.0,20,175.299988,390,0.000053


year                 0
state                0
registration         0
population           0
reg_per_capita       0
affordability        0
incentives           0
renewable_prod       5
charge_stations      0
charge_per_capita    0
dtype: int64


## Preregistration Statement: 

### Preregistration Statement 1

- **Hypothesis**:  
  State-level electric vehicle (EV) registrations increase with the total number of federal incentives available.

- **Context**:  
  Federal incentives, such as tax credits and subsidies, are designed to reduce the upfront cost of EVs, making them more accessible to consumers. Studies have shown that these incentives significantly influence consumer purchasing decisions, leading to higher EV adoption rates. For instance, research indicates that federal tax credits have been a major driver in the growth of the EV market in the United States [1].

- **Analysis**:  
  To test this hypothesis, we will run a linear regression per state with the total count of federal EV incentives as the independent variable and the total EV registrations in the state as the dependent variable. We will test whether βfederal_incentives > 0 for each model, which would indicate that an increase in total federal incentives is associated with increased total EV registrations at the state level.  
---

### Preregistration Statement 2

- **Hypothesis**:  
  State-level EV registrations increase with the state's renewable energy production.

- **Context**:  
  States that produce more renewable energy may foster a culture of environmental consciousness, leading to higher EV adoption rates. Additionally, consumers in these states might prefer EVs to align with the clean energy available, reducing the overall carbon footprint of their transportation. Research has shown a correlation between renewable energy policies and increased EV adoption, suggesting that the availability of clean energy sources can influence consumer behavior [2].

- **Analysis**:  
  For this hypothesis, we will run a linear regression per state with the renewable energy production of the state as the independent variable and the total EV registrations in the state as the dependent variable. We will test whether βrenewable_energy > 0 for each model, which would indicate that an increase in a state’s renewable energy production is associated with increased total EV registrations of the state.  

---

These analyses aim to elucidate the impact of federal incentives and renewable energy production on EV adoption at the state level, providing insights for policymakers to design effective strategies to promote sustainable transportation.

---

### Citations

1. Javadnejad et al. (2023). *Federal tax credits and EV adoption*. [https://link.springer.com/article/10.1007/s10669-023-09958-3](https://link.springer.com/article/10.1007/s10669-023-09958-3)  
2. Pabitra Kumar Das & Mohammad Younus Bhat. (2021). *Renewable energy policies and EV adoption*. [https://link.springer.com/article/10.1007/s11356-021-18211-w](https://link.springer.com/article/10.1007/s11356-021-18211-w)


## Data Analysis: 

**Regression Model**

In [4]:
data = data.set_index(['state', 'year'])

input_var = ['affordability', 'incentives', 'renewable_prod', 'charge_stations']
X = data[input_var]
X = sm.add_constant(X)
y = data['registration']

# Fit PanelOLS with fixed effects
model = PanelOLS(y, X, entity_effects=True).fit()
print(model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:           registration   R-squared:                        0.7661
Estimator:                   PanelOLS   R-squared (Between):              0.9064
No. Observations:                  35   R-squared (Within):               0.7661
Date:                Wed, Nov 20 2024   R-squared (Overall):              0.8524
Time:                        20:54:37   Log-likelihood                   -431.73
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      21.294
Entities:                           5   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                    F(4,26)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             21.294
                            

/Users/jamiejeong/opt/anaconda3/envs/info2950/lib/python3.12/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [5]:
# Charge stations per capita
input_var = ['affordability', 'incentives', 'renewable_prod', 'charge_per_capita']
X = data[input_var]
X = sm.add_constant(X)

# Per Capita
y = data['reg_per_capita']

# Fit PanelOLS with fixed effects
model = PanelOLS(y, X, entity_effects=True).fit()
print(model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:         reg_per_capita   R-squared:                        0.9022
Estimator:                   PanelOLS   R-squared (Between):              0.7410
No. Observations:                  35   R-squared (Within):               0.9022
Date:                Wed, Nov 20 2024   R-squared (Overall):              0.8352
Time:                        20:56:53   Log-likelihood                    187.21
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      59.992
Entities:                           5   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                    F(4,26)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             59.992
                            

/Users/jamiejeong/opt/anaconda3/envs/info2950/lib/python3.12/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


## Evaluation of Significance: 

## Conclusions: 

## Limitations: 

## Acknowledgements and Bibliography: 

<p style="padding-left: 40px; text-indent: -40px;">
U.S. Census Bureau, Population Division. (2019). Annual Estimates of the Resident Population for the United States, Regions, States, and Puerto Rico: April 1, 2010 to July 1, 2019 (NST-EST2019-01) [Data file]. Retrieved from https://www2.census.gov/programs-surveys/popest/tables/2010-2019/state/totals/nst-est2019-01.xlsx
</p>

<p style="padding-left: 40px; text-indent: -40px;">
U.S. Census Bureau, Population Division. (2023). Annual Estimates of the Resident Population for the United States, Regions, States, District of Columbia, and Puerto Rico: April 1, 2020 to July 1, 2023 (NST-EST2023-POP) [Data file]. Retrieved from https://www2.census.gov/programs-surveys/popest/tables/2020-2023/state/totals/NST-EST2023-POP.xlsx
</p>


<p style="padding-left: 40px; text-indent: -40px;">
U.S. Department of Energy. (2023). Alternative Fuels Data Center: Vehicle Registration Data. Retrieved from https://afdc.energy.gov/vehicle-registration?year=2023
</p>

<p style="padding-left: 40px; text-indent: -40px;">
U.S. Energy Information Administration. (n.d.). State Energy Data System (SEDS): 1960-2021 (Complete) [Data file]. Retrieved from https://www.eia.gov/state/seds/sep_prod/SEDS_Production_Report.pdf
</p>

<p style="padding-left: 40px; text-indent: -40px;">
U.S. Department of Energy. (n.d.). Find and Compare Cars. Retrieved from https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2023&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&cbvtelectric=Electric&YearSel=2016-2023&make=&mclass=&vfuel=&vtype=Electric&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200&pageno=1&tabView=0
</p>

<p style="padding-left: 40px; text-indent: -40px;">
Open Charge Map. (2024). Open Charge Map API. Retrieved November 20, 2024, from https://openchargemap.org/site/develop/api
</p>

<p style="padding-left: 40px; text-indent: -40px;">
U.S. Department of Energy, Office of Energy Efficiency & Renewable Energy. (2024). Alternative Fuels Data Center: Data Download [Data file]. Retrieved November 20, 2024, from https://afdc.energy.gov/data_download
</p>

<p style="padding-left: 40px; text-indent: -40px;">
Javadnejad et al. (2023). Federal tax credits and EV adoption. https://link.springer.com/article/10.1007/s10669-023-09958-3
</p>

<p style="padding-left: 40px; text-indent: -40px;">
Pabitra Kumar Das & Mohammad Younus Bhat. (2021). Renewable energy policies and EV adoption. https://link.springer.com/article/10.1007/s11356-021-18211-w  
</p>